In [6]:
import pandas as pd

In [7]:
RAW_DATA_PATH = './Dataset/'
CLEANED_DATA_PATH = './Dataset/Cleaned/'
WORLDCUP_2010 = 'raw_worldcup_2010.csv'
WORLDCUP_2014 = 'raw_worldcup_2014.csv'
WORLDCUP_2018 = 'raw_worldcup_2018.csv'

In [175]:
def clean_worldcup_data(raw_data_path, raw_data):
    
    # read raw csv data and fill 0 if it's N/A
    df = pd.read_csv(raw_data_path + raw_data)
    df = df.fillna(0)
    
    if raw_data == 'raw_worldcup_2010.csv':
        df['YEAR'] = 2010
    elif raw_data == 'raw_worldcup_2014.csv':
        df['YEAR'] = 2014
    elif raw_data == 'raw_worldcup_2018.csv':
        df['YEAR'] = 2018
        df['A_L'] = df['A_Z1']
        df['A_M'] = df['A_Z2'] + df['A_Z3']
        df['A_H'] = df['A_Z4'] + df['A_Z5']
    else:
        df['YEAR'] = 0

    column_filter = ['YEAR','MATCH', 'TEAM', 'POS', 'PC ', 'PA ', 'PC/PA', 'GS', 'S', 
                     'SG', 'FC', 'FS', 'DC', 'D_IN_POSS', 'D_NOT_IN_POSS',
                     'T_OPP_HALF', 'T_ATT_3RD', 'T_PEN_AREA','SPRINT',
                     'MAX_SPEED', 'A_L', 'A_M', 'A_H', 'Saves','Yellow_cards', 'Red_card', 
                     'Tackles_gaining_the_ball', 'Tackles_suffered_losing_the_ball']
    
    df = df[column_filter]
    
    # append the missing value of fowards in year 2010 with means of the columns
    if raw_data == 'raw_worldcup_2010.csv':
        df2 = pd.DataFrame({
            'YEAR':2010, 'MATCH':23, 'TEAM':'ALG', 'POS':'F', 'PC ':df['PC '].mean(), 'PA ':df['PA '].mean(), 
            'PC/PA':df['PC/PA'].mean(), 'GS':0, 'S':df['S'].mean(), 
            'SG':df['SG'].mean(), 'FC':df['FC'].mean(), 'FS':df['FS'].mean(), 'DC':df['DC'].mean(), 
            'D_IN_POSS':df['D_IN_POSS'].mean(), 'D_NOT_IN_POSS':df['D_NOT_IN_POSS'].mean(),
            'T_OPP_HALF':df['T_OPP_HALF'].mean(), 'T_ATT_3RD':df['T_ATT_3RD'].mean(), 'T_PEN_AREA':df['T_PEN_AREA'].mean(),
            'SPRINT':df['SPRINT'].mean(),'MAX_SPEED':df['MAX_SPEED'].mean(), 'A_L':df['A_L'].mean(), 'A_M':df['A_M'].mean(), 
            'A_H':df['A_H'].mean(), 'Saves':0,'Yellow_cards':df['Yellow_cards'].mean(), 'Red_card':df['Red_card'].mean(), 
            'Tackles_gaining_the_ball':df['Tackles_gaining_the_ball'].mean(), 'Tackles_suffered_losing_the_ball':df['Tackles_suffered_losing_the_ball'].mean()
        }, index = [180])
        df = df.append(df2)
    
    match_result = df[['MATCH','TEAM','GS','YEAR']]
    match_result = match_result.groupby(['MATCH','TEAM']).aggregate({'MATCH':'first','TEAM':'first','GS':'sum'})
    
    match_result['result']=0 # 1:win 0: draw -1: lose
    
    for i in range(1,len(match_result)//2+1):
        score = match_result.loc[i,:]['GS']
        result = match_result.loc[i,:]['result']
        
        if (score[0] > score[1]):
            result[0] = 1
            result[1] = -1
        elif (score[0] < score[1]):
            result[0] = -1
            result[1] = 1
    
    match_result = match_result.rename(columns={
        'MATCH':'match',
        'TEAM':'team',
        'GS':'goals'
    })
    
    
    aggregate_method = {}
    for f in column_filter:
        if f=='MATCH' or f=='TEAM' or f=='POS' or f=='YEAR':
            aggregate_method[f] = 'first'
        elif f=='PC/PA' or f=='A_L' or f=='A_M' or f=='A_H' or f=='T_OFF_HALF' or f=='T_ATT_3RD' or f=='T_PEN_AREA' or f=='MAX_SPEED':
            aggregate_method[f] = 'mean'
        else:
            aggregate_method[f] = 'sum'
            
    
    df = df.groupby(['YEAR','MATCH','TEAM', 'POS']).aggregate(aggregate_method)
    
    df_gk = pd.DataFrame(columns=['GK_passes_succeed', 'GK_total passes', 'GK_passes accuracy', 'GK_goals', 'GK_total shots', 
                     'GK_on-target', 'GK_fouls committed', 'GK_fouls suffered', 'GK_distance', 'GK_distance in poss', 'GK_distance not in poss',
                     'GK_time on opposite half', 'GK_time on opposite third', 'GK_time on opposite pen','GK_sprints',
                     'GK_avg max speed', 'GK_low activity', 'GK_mid activity', 'GK_high activity', 'GK_saves','GK_yellow', 'GK_red', 
                     'GK_tackles gaining ball', 'GK_tackles suffered losing ball'])
    df_df = pd.DataFrame(columns=['DF_passes_succeed', 'DF_total passes', 'DF_passes accuracy', 'DF_goals', 'DF_total shots', 
                     'DF_on-target', 'DF_fouls committed', 'DF_fouls suffered', 'DF_distance', 'DF_distance in poss', 'DF_distance not in poss',
                     'DF_time on opposite half', 'DF_time on opposite third', 'DF_time on opposite pen','DF_sprints',
                     'DF_avg max speed', 'DF_low activity', 'DF_mid activity', 'DF_high activity', 'DF_saves','DF_yellow', 'DF_red', 
                     'DF_tackles gaining ball', 'DF_tackles suffered losing ball'])
    df_mf = pd.DataFrame(columns=['MF_passes_succeed', 'MF_total passes', 'MF_passes accuracy', 'MF_goals', 'MF_total shots', 
                     'MF_on-target', 'MF_fouls committed', 'MF_fouls suffered', 'MF_distance', 'MF_distance in poss', 'MF_distance not in poss',
                     'MF_time on opposite half', 'MF_time on opposite third', 'MF_time on opposite pen','MF_sprints',
                     'MF_avg max speed', 'MF_low activity', 'MF_mid activity', 'MF_high activity', 'MF_saves','MF_yellow', 'MF_red', 
                     'MF_tackles gaining ball', 'MF_tackles suffered losing ball'])
    df_fw = pd.DataFrame(columns=['FW_passes_succeed', 'FW_total passes', 'FW_passes accuracy', 'FW_goals', 'FW_total shots', 
                     'FW_on-target', 'FW_fouls committed', 'FW_fouls suffered', 'FW_distance', 'FW_distance in poss', 'FW_distance not in poss',
                     'FW_time on opposite half', 'FW_time on opposite third', 'FW_time on opposite pen','FW_sprints',
                     'FW_avg max speed', 'FW_low activity', 'FW_mid activity', 'FW_high activity', 'FW_saves','FW_yellow', 'FW_red', 
                     'FW_tackles gaining ball', 'FW_tackles suffered losing ball'])
    for index, row in df.iterrows():
        if row['POS'] == 'GK' or row['POS'] == 'GK ':
            df_gk.loc[len(df_gk.index)] = row.values.tolist()[4:]
        elif row['POS'] == 'DF' or row['POS'] == 'D' or row['POS'] == 'D ':
            df_df.loc[len(df_df.index)] = row.values.tolist()[4:]
        elif row['POS'] == 'MF' or row['POS'] == 'M' or row['POS'] == 'M ':
            df_mf.loc[len(df_mf.index)] = row.values.tolist()[4:]
        elif row['POS'] == 'FW' or row['POS'] == 'F' or row['POS'] == 'F ':
            df_fw.loc[len(df_fw.index)] = row.values.tolist()[4:]
    
    match_result = match_result.reset_index(level=['MATCH', 'TEAM'])    
    data_sets = [match_result, df_gk, df_df, df_mf, df_fw]
    
    df = pd.concat(data_sets, axis=1)
    
    # changes the order of columns so the result goes to the end
    columns = df.columns.tolist()
    result = columns[5]
    columns = columns[2:5] + columns[6:]
    columns.append(result)
    df = df[columns]
    df = df.drop(columns=['GK_goals', 'DF_goals','MF_goals', 'FW_goals'])
    
    return df
    
    

In [167]:
def store_cleaned_data(path, game, year):
    game_file = 'cleaned_game_' + str(year) + '.csv' 
    game.to_csv(path+game_file)

In [176]:
import warnings
warnings.filterwarnings('ignore')

game_2010 = clean_worldcup_data(RAW_DATA_PATH, WORLDCUP_2010)
game_2014 = clean_worldcup_data(RAW_DATA_PATH, WORLDCUP_2014)
game_2018 = clean_worldcup_data(RAW_DATA_PATH, WORLDCUP_2018)


In [177]:
store_cleaned_data(CLEANED_DATA_PATH,game_2010, 2010)
store_cleaned_data(CLEANED_DATA_PATH,game_2014, 2014)
store_cleaned_data(CLEANED_DATA_PATH,game_2018, 2018)